# Convert CSV to JSON
- JSON files are smaller and faster to read than CSVs
- Plus CSV image lists have to be converted from string to list after reading in
- I can also convert floats to ints as the extra precision doesn't add anything

In [18]:
%matplotlib inline
#from pylab import rcParams
#rcParams['figure.figsize'] = 12, 8

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information numpy, scipy, matplotlib, pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 2.7.12 64bit [GCC 5.4.0 20160609]
IPython 5.4.1
OS Linux 4.10.0 32 generic x86_64 with Ubuntu 16.04 xenial
numpy 1.13.1
scipy 0.19.1
matplotlib 2.0.2
pandas 0.20.3
Sun Aug 20 19:40:58 2017 AEST

In [19]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import cv2
import pandas as pd
import uuid

import os
import sys
import glob
import multiprocessing

sys.path.append("../src")

np.random.seed(0)

def view(image):
    plt.figure()
    plt.axis('off')
    plt.imshow(image, cmap = cm.Greys_r)

In [20]:
def read_coordinates_file(coord_file_path):
    file_extension = coord_file_path.split('.')[-1]
    if file_extension == 'csv':
        coords_data = pd.read_csv(coord_file_path)
        coords_data['flattened_28x28_tag_matrices'] = coords_data['flattened_28x28_tag_matrices'].apply(lambda x: eval(x) if x is not np.nan else None)
    elif file_extension == 'json':
        with open(coord_file_path) as json_file:
            coords_data = json.load(json_file)
    else:
        print('Unknown coordinate file type - not csv or json')
        sys.exit(0)

    return (coords_data, file_extension)

In [21]:
def convert_csv_json(csv_file_path):
    video_filename = get_video_filename(csv_file_path)
    
    df, extension = read_coordinates_file(csv_file_path)
    df = df.sort_values('frame_nums', ascending=True)
    df['x'] = df['x'].astype(int)
    df['y'] = df['y'].astype(int)
    
    df.to_json('/home/jack/Data/Caffeine_Unique_Tags/raw/' + video_filename + '.json', orient='records')

def get_video_filename(video_path):
    head, tail = os.path.split(video_path)
    str_date_time = os.path.splitext(tail)[0]
    return str_date_time

In [29]:
csv_file_list = glob.glob('/media/jack/JSIMPSON/1b/csv/*.csv')
print(csv_file_list[0], len(csv_file_list))

unprocessed_csv_file_list = []
for c in csv_file_list:
    video_filename = get_video_filename(c)
    new_csv_filename = '/home/jack/Data/Caffeine_Unique_Tags/raw/' + video_filename + '.json'
    if not os.path.exists(new_csv_filename):
        unprocessed_csv_file_list.append(c)

print(len(unprocessed_csv_file_list), unprocessed_csv_file_list[0])

('/media/jack/JSIMPSON/1b/csv/2017-03-02_08-03-32.csv', 29)
(1, '/media/jack/JSIMPSON/1b/csv/2017-03-02_17-03-39.csv')


In [30]:
processes = multiprocessing.Pool(processes=3)
frame_data_list = processes.map(func=convert_csv_json, iterable=csv_file_list, chunksize=1)